In [1]:
import allel
print(allel.__version__)

1.2.1


In [2]:
callset = allel.read_vcf('../1000genomes_samples.vcf')

In [3]:
callset.keys()

dict_keys(['samples', 'calldata/GT', 'variants/ALT', 'variants/CHROM', 'variants/FILTER_PASS', 'variants/ID', 'variants/POS', 'variants/QUAL', 'variants/REF'])

In [4]:
len(callset["calldata/GT"])

68818

In [5]:
callset["variants/CHROM"][40:80]

array(['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1'], dtype=object)

In [6]:
callset["variants/POS"][40:80]

array([1609210, 1626665, 1627805, 1649047, 1649620, 1772216, 1922946,
       2024547, 2028833, 2162106, 2303828, 2372200, 2374226, 2397655,
       2407712, 2524546, 2623324, 2630903, 2684305, 2693016, 2701516,
       2704855, 2813287, 2813548, 2847963, 2848661, 2911536, 2918659,
       2920631, 2937832, 3009447, 3011887, 3063073, 3098371, 3144724,
       3186949, 3209922, 3481851, 3488273, 3508964], dtype=int32)

In [7]:
len(callset["calldata/GT"][41])

2504

In [8]:
from pyensembl import EnsemblRelease

In [9]:
data = EnsemblRelease(77)

In [35]:
genes = data.genes_at_locus(contig=1, position=3508964)

In [37]:
genes

[Gene(gene_id='ENSG00000162591', gene_name='MEGF6', biotype='protein_coding', contig='1', start=3489920, end=3611495, strand='-', genome='GRCh38')]

In [24]:
genes[0].gene_id

'ENSG00000162591'

In [30]:
gene_ids = []
for i in range(len(callset["variants/POS"])):
    position = int(callset["variants/POS"][i])
    chromosome = callset["variants/CHROM"][i]
    if chromosome != 'X':
        genes = data.genes_at_locus(contig=int(chromosome), position=position)
        gene_ids += ([gene.gene_id for gene in genes])
    else:
        genes = data.genes_at_locus(contig=23, position=position)
        gene_ids += ([gene.gene_id for gene in genes])


In [78]:
len(gene_ids)

1

In [79]:
len(set(gene_ids))

1

In [34]:
import pandas as pd

gene_encodings_df = pd.read_csv("../Gene_freq_encoding.csv")

In [36]:
gene_encodings_df.head()

,Unnamed: 0,DonorIDs,CancerType,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,...,ENSG00000273456,ENSG00000273466,ENSG00000273471,ENSG00000273476,ENSG00000273477,ENSG00000273478,ENSG00000273483,ENSG00000273485,ENSG00000273488,ENSG00000273489
0,0,DO48566BLCA-US,BLCA-US,0.0,0.0,0.0,2.0,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,DO223588BLCA-US,BLCA-US,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2,DO51948BLCA-US,BLCA-US,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,DO514BLCA-US,BLCA-US,0.0,0.0,0.0,0.0,1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,DO474BLCA-US,BLCA-US,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
gene_labels = [label for label in gene_encodings_df.columns if label.startswith("ENSG")]

In [90]:
gene_encodings_df.columns

Index(['Unnamed: 0', 'DonorIDs', 'CancerType', 'ENSG00000000003',
       'ENSG00000000005', 'ENSG00000000419', 'ENSG00000000457',
       'ENSG00000000460', 'ENSG00000000938', 'ENSG00000000971',
       ...
       'ENSG00000273456', 'ENSG00000273466', 'ENSG00000273471',
       'ENSG00000273476', 'ENSG00000273477', 'ENSG00000273478',
       'ENSG00000273483', 'ENSG00000273485', 'ENSG00000273488',
       'ENSG00000273489'],
      dtype='object', length=30546)

In [75]:
gene_labels = set(gene_labels)

In [80]:
len(gene_labels)

30543

In [76]:
healthy_gene_labels = set(gene_ids)

In [77]:
len(gene_labels.intersection(healthy_gene_labels))

1

In [81]:
healthy_sample_ids = [
    i for i in range(gene_encodings_df.shape[0], gene_encodings_df.shape[0] + len(callset["samples"]))
]

In [82]:
healthy_donor_ids = ["N/A" for i in range(len(callset["samples"]))]

In [83]:
healthy_cts = ["Healthy" for i in range(len(callset["samples"]))]

In [84]:
import numpy as np

In [85]:
healthy_genes_affected = {}
        
for gene_label in gene_labels:
    healthy_genes_affected[gene_label] = np.zeros(len(callset["samples"]))

In [86]:
for i in range(len(callset["variants/POS"])):
    position = int(callset["variants/POS"][i])
    chromosome = callset["variants/CHROM"][i]
    calldata = np.max(callset["calldata/GT"][i], axis=-1)
    genes = []
    if chromosome != 'X':
        genes += data.genes_at_locus(contig=int(chromosome), position=position)   
    else:
        genes += data.genes_at_locus(contig=23, position=position)
    if len(genes) > 0:
        gene_ids = [gene.gene_id for gene in genes]
        for gene_id in gene_ids:
            if gene_id in gene_labels:
                healthy_genes_affected[gene_id] += calldata

In [87]:
healthy_genes_affected

{'ENSG00000152503': array([1., 1., 1., ..., 1., 1., 1.]),
 'ENSG00000238369': array([0., 0., 0., ..., 0., 0., 0.]),
 'ENSG00000255215': array([0., 0., 0., ..., 0., 0., 0.]),
 'ENSG00000119711': array([0., 0., 0., ..., 0., 0., 0.]),
 'ENSG00000155729': array([0., 0., 0., ..., 0., 0., 0.]),
 'ENSG00000162745': array([0., 0., 0., ..., 0., 0., 0.]),
 'ENSG00000235482': array([0., 0., 0., ..., 0., 0., 0.]),
 'ENSG00000231233': array([0., 0., 0., ..., 0., 0., 0.]),
 'ENSG00000240713': array([0., 0., 0., ..., 0., 0., 0.]),
 'ENSG00000230498': array([0., 0., 0., ..., 0., 0., 0.]),
 'ENSG00000196267': array([0., 0., 0., ..., 0., 0., 0.]),
 'ENSG00000212066': array([0., 0., 0., ..., 0., 0., 0.]),
 'ENSG00000267838': array([0., 0., 0., ..., 0., 0., 0.]),
 'ENSG00000066697': array([0., 0., 0., ..., 0., 0., 0.]),
 'ENSG00000114107': array([0., 0., 0., ..., 0., 0., 0.]),
 'ENSG00000207241': array([0., 0., 0., ..., 0., 0., 0.]),
 'ENSG00000265367': array([0., 0., 0., ..., 0., 0., 0.]),
 'ENSG00000263

In [91]:
healthy_df = pd.DataFrame()

In [94]:
healthy_df['Unnamed: 0'] = healthy_sample_ids

In [96]:
healthy_df['DonorIDs'] = healthy_donor_ids

In [97]:
healthy_df['CancerType'] = healthy_cts

In [99]:
cancer_df_genes = [label for label in gene_encodings_df.columns if label.startswith("ENSG")]

In [100]:
for gene_label in cancer_df_genes:
    healthy_df[gene_label] = healthy_genes_affected[gene_label]

In [101]:
healthy_df

,Unnamed: 0,DonorIDs,CancerType,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,...,ENSG00000273456,ENSG00000273466,ENSG00000273471,ENSG00000273476,ENSG00000273477,ENSG00000273478,ENSG00000273483,ENSG00000273485,ENSG00000273488,ENSG00000273489
0,3364,N/A,Healthy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3365,N/A,Healthy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3366,N/A,Healthy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3367,N/A,Healthy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3368,N/A,Healthy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499,5863,N/A,Healthy,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2500,5864,N/A,Healthy,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2501,5865,N/A,Healthy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2502,5866,N/A,Healthy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
complete_data = pd.concat([gene_encodings_df, healthy_df])

In [103]:
complete_data

,Unnamed: 0,DonorIDs,CancerType,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,...,ENSG00000273456,ENSG00000273466,ENSG00000273471,ENSG00000273476,ENSG00000273477,ENSG00000273478,ENSG00000273483,ENSG00000273485,ENSG00000273488,ENSG00000273489
0,0,DO48566BLCA-US,BLCA-US,0.0,0.0,0.0,2.0,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,DO223588BLCA-US,BLCA-US,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2,DO51948BLCA-US,BLCA-US,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,DO514BLCA-US,BLCA-US,0.0,0.0,0.0,0.0,1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,DO474BLCA-US,BLCA-US,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499,5863,N/A,Healthy,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2500,5864,N/A,Healthy,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2501,5865,N/A,Healthy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2502,5866,N/A,Healthy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
complete_data.to_csv("Merged_Gene_freq_encoding.csv")

In [105]:
healthy_df.to_csv("Healthy_Gene_freq_encoding.csv")